Package location on my system:

Works with conda environment for recommender systems by Python 3.10

## Collaborative Filtering
#### Model Based Approach

In [1]:
import pandas as pd

from surprise import SVD
from surprise import Dataset
from surprise import Reader

from surprise import accuracy

from surprise.model_selection import train_test_split

from surprise.model_selection import GridSearchCV

from surprise.model_selection import cross_validate

We will be working with the [same data](https://drive.google.com/file/d/1WvTmAfO09TCX7xp7uu06__ziic7JnrL5/view?usp=sharing) we used in the previous exercise.

In [2]:
book_ratings = pd.read_csv('BX-CSV-Dump\\BX-Book-Ratings.csv',sep=";", encoding="latin")

In [3]:
book_ratings.head(5)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [4]:
book_ratings.groupby('Book-Rating').describe()

User-ID                                                 \
                count           mean           std     min       25%   
Book-Rating                                                            
0            716109.0  143370.549256  80345.143780     2.0  73394.00   
1              1770.0  127012.059322  86750.741252  1314.0  48423.75   
2              2759.0  138600.495469  84060.292551   387.0  62464.00   
3              5996.0  138070.616077  83818.299314    17.0  63180.25   
4              8904.0  138104.758760  83264.004877    86.0  65514.00   
5             50974.0  137953.412544  79099.693575     8.0  70183.00   
6             36924.0  135065.780820  81575.208656     8.0  64015.00   
7             76457.0  134472.395163  81432.702382     8.0  63694.00   
8            103736.0  135406.822010  80778.299921    32.0  67542.00   
9             67541.0  135306.242638  79049.826965    16.0  70594.00   
10            78610.0  134765.543213  80929.515759    12.0  63714.00   

                                            
                  50%        75%       max  
Book-Rating                                 
0            145641.0  212898.00  278854.0  
1            116395.5  208668.00  278759.0  
2            138018.0  216626.00  278764.0  
3            137649.0  214553.75  278820.0  
4            137190.0  213316.00  278723.0  
5            138777.0  202200.25  278851.0  
6            135149.0  206023.00  278854.0  
7            133571.0  205485.00  278854.0  
8            133123.0  206074.00  278854.0  
9            129465.0  203611.00  278849.0  
10           130081.0  206438.00  278851.0

In [5]:
book_ratings['Book-Rating'].describe()

count    1.149780e+06
mean     2.866950e+00
std      3.854184e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      7.000000e+00
max      1.000000e+01
Name: Book-Rating, dtype: float64

* create surprise dataset from book_ratings

In [6]:
reader = Reader(rating_scale=(0, 1))

# Loads Pandas dataframe
data = Dataset.load_from_df(book_ratings, reader)

* split data to train and test set, use test size 15%

In [7]:
train_set, test_set = train_test_split(data, test_size=0.15) # <<--the best 0.3

In [8]:
reader.rating_scale

(0, 1)

* Use SVD (with default settings) to create recommendations for each user
    - print default model's rmse that was computed on the test set (using object accuracy we imported in the beginning)

In [9]:
# Breaking the matrix into 3 smaller matrices
model = SVD()

In [10]:
model.fit(train_set)

In [11]:
predictions = model.test(test_set)

In [12]:
accuracy.rmse(predictions)

RMSE: 4.2892


4.289241063354694

<font color= 'red'> Description: </font>
<font color= 'Orange'> Usually RMSE is expected to be between 0 and 1, while in this case it is not. If we look at the upper section's describe() results, we would see the count of the USER-IDs which have got ratings 0-10, total count, mean value, and standard deviation; we can notice such result is not that much bad </font>

<font color='orange'> Train test set splitting for grid search </font>

* create parameters grid, use this params:
* 'n_factors': [110, 120, 140, 160]
* 'reg_all': [0.08, 0.1, 0.15]

In [13]:
param_grid = {'n_factors': [160, 180, 200], 'reg_all': [0.15, 0.2, 0.3]}

* instantiate GridSearch with SVD as model, our pre-defined parameter grid and rmse and mae as evaluation metrics

In [14]:
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=4)

* fit GridSearch

In [15]:
grid_search.fit(data)

* print best RMSE score from training

In [16]:
print(grid_search.best_score['rmse'])
print(grid_search.best_params['rmse'])

4.277259958594473
{'n_factors': 200, 'reg_all': 0.15}


In [17]:
best_model = SVD(n_factors=200, reg_all=0.15)

In [18]:
best_model.fit(train_set)

* predict test set with optimal model based on `RMSE`

In [19]:
predictions = best_model.test(test_set)

In [20]:
accuracy.rmse(predictions)

RMSE: 4.2822


4.282164634475405

* print optimal model's RMSE that was computed on test set
    - is it better than the default parameters?

In [22]:
percentage = (1-4.2772/4.2822)*100
print(f'{percentage:.3f}%')

0.117%


The optimal model only have less than 1% improvement. So, it can be avoided.